In [1]:
import os

from dataclasses import dataclass
from pathlib import Path
from textsummarizer.utils.common import read_yaml, create_directories
from textsummarizer.constants import *
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from textsummarizer.loggin import logger
from datasets import load_dataset, load_from_disk
import torch

In [4]:
@dataclass(frozen = True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    per_device_eval_batch_size: int
    weight_decay: float
    logging_steps: int
    eval_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

In [10]:
class Configurationmanager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        
            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)

            create_directories([self.config.artifacts_root])

    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
          config = self.config.model_trainer
          params = self.params.TrainingArguments


          create_directories([config.root_dir])

          model_trainer_config = ModelTrainerConfig(
              root_dir = config.root_dir,
              data_path = config.data_path,
              model_ckpt = config.model_ckpt,
              num_train_epochs = params.num_train_epochs,
              warmup_steps = params.warmup_steps,
              per_device_train_batch_size = params.per_device_train_batch_size,
              per_device_eval_batch_size = params.per_device_eval_batch_size,
              weight_decay = params.weight_decay,
              logging_steps = params.logging_steps,
              eval_strategy = params.eval_strategy,
              eval_steps = params.eval_steps,
              save_steps = params.save_steps,
              gradient_accumulation_steps = params.gradient_accumulation_steps
                )

          return model_trainer_config

In [9]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config


    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model = model_pegasus)

        #loading data
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        trainer_args = TrainingArguments(
            output_dir = self.config.root_dir, num_train_epochs= self.config, warmup_steps=self.config.warmup_steps,
            per_device_train_batch_size=self.config.per_device_train_batch_size, per_device_eval_batch_size=self.config.per_device_eval_batch_size,
            weight_decay=self.config.weight_decay, logging_steps=self.config.logging_steps,
            evaluation_strategy=self.config.evaluation_strategy, eval_steps=self.config.eval_steps, save_steps=self.config.save_steps,
            gradient_accumulation_steps= self.config.gradient_accumulation_steps
        )


        trainer = Trainer(model=model_pegasus, args=trainer_args,
                    tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                    train_dataset=dataset_samsum_pt["train"],
                    eval_dataset=dataset_samsum_pt["validation"],
                    )
        
        trainer.train()

        # save model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir,"pegasus-samsum-model"))
        # save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))


In [11]:
try:
    config = Configurationmanager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-07-12 15:17:40,303: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-12 15:17:40,632: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-12 15:17:40,684: INFO: common: Created directory: artifacts]
[2024-07-12 15:17:40,847: INFO: common: Created directory: artifacts/model_trainer]



KeyboardInterrupt

